# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [22]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


### Importing the libraries

In [23]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [24]:
class Network(nn.Module):
#nn.Module is the base class for all neural network modules in PyTorch.
#By inheriting from it, Network gains access to methods and properties
#that help in constructing and managing neural networks,
#such as parameter management and model serialization.

  def __init__(self, state_size, action_size, seed = 42): #The __init__ method is the constructor that gets called when an instance of Network is created
    super(Network, self).__init__() # calls the constructor of the parent class (nn.Module) to properly initialize the base class parts.
    self.seed = torch.manual_seed(seed) #torch.manual_seed(seed) sets the seed for generating random numbers, which ensures that the model's initialization and subsequent operations are deterministic (reproducible) given the same seed.
    #self.seed stores this seed value, which might be useful later in the code for debugging or reproducibility purposes.
    self.fc1 = nn.Linear(state_size, 64)
    #This creates the first fully connected layer (fc1).
    #It takes the input of size state_size and outputs a tensor of size 64.
    #nn.Linear applies a linear transformation to the incoming data: y = xW^T + b,
    #where W is the weight matrix and b is the bias
    self.fc2 = nn.Linear(64, 64)
    #This creates the second fully connected layer (fc2).
    #It takes the output from the first layer, which is of size 64, and produces another output of size 64.
    self.fc3 = nn.Linear(64, action_size)
    #This creates the third fully connected layer (fc3).
    #It takes the output from the second layer (size 64) and produces an output of size action_size.
    #This layer outputs the final predictions, typically representing the action values in a reinforcement learning context (like Q-values in Q-learning).
  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

#The forward method defines the forward pass of the network, i.e., how the input state passes through the network layers to produce the output.

# x = self.fc1(state)

# The input state (a tensor representing the state) is passed through the first fully connected layer (fc1).
# The output x is a tensor of size 64.
# x = F.relu(x)

# The output from fc1 is passed through a ReLU (Rectified Linear Unit) activation function.
# F.relu applies the ReLU function element-wise: ReLU(x) = max(0, x).
# This introduces non-linearity into the model, enabling it to learn more complex patterns.
# x = self.fc2(x)

# The output from the ReLU activation is passed through the second fully connected layer (fc2).
# The output x is again a tensor of size 64.
# x = F.relu(x)

# Similar to the first ReLU, this applies the ReLU activation to the output of the second layer, introducing another layer of non-linearity.
# return self.fc3(x)

# Finally, the output from the second ReLU is passed through the third fully connected layer (fc3).
# This produces the final output, a tensor of size action_size.
# This output represents the predicted values for each possible action in a reinforcement learning scenario.

## Part 2 - Training the AI

### Setting up the environment

In [25]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [26]:
learning_rate = 5e-4 #The learning rate is a crucial hyperparameter in the training of neural networks, representing the step size at each iteration while moving toward a minimum of the loss function.

minibatch_size = 100 #The minibatch size is the number of samples that will be processed together before updating the model's parameters.

discount_factor = 0.99 #The discount factor is used in the calculation of the expected return, which guides the learning process. For instance, in Q-learning, it influences the value of the Q-function by weighing future rewards.

replay_buffer_size = int(1e5) #the replay buffer can store up to 100,000 experiences.
#The replay buffer helps break the correlation between consecutive experiences by sampling random minibatches during training, which leads to more stable learning.

interpolation_parameter = 1e-3 # implies that the target network parameters are updated very slowly, using a small proportion of the main network parameters at each step.

### Implementing Experience Replay

In [27]:
class ReplayMemory(object):
# ReplayMemory class is designed to store and manage experiences (tuples of state, action, reward, next state, and done flag) for use in reinforcement learning,
# particularly in algorithms like Deep Q-Networks (DQN).
  def __init__(self, capacity): # This is the constructor method that initializes an instance of the ReplayMemory class.
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # This line checks if a GPU is available (torch.cuda.is_available()).
    # If a GPU is available, it sets the self.device to "cuda:0", otherwise, it defaults to the CPU ("cpu").
    # This device setting is crucial for ensuring that the tensors (data) are stored and processed on the correct device (CPU or GPU).
    self.capacity = capacity
    # It defines the maximum number of experiences that the memory can hold.
    # self.capacity stores this value, which will be used to manage the size of the memory.
    self.memory = []
    # This list will store the experiences (state, action, reward, next state, done) as they are added.
    # The experiences are stored as tuples within this list.


  def push(self, event): #This method is used to add a new experience to the memory.
    self.memory.append(event)
    # The event (which is a tuple containing state, action, reward, next state, and done) is appended to the self.memory list.
    # This action adds the latest experience to the replay memory.
    if len(self.memory) > self.capacity: #This conditional checks if the size of self.memory exceeds the predefined capacity.
      del self.memory[0]
    # If the memory exceeds its capacity, the oldest experience (the one at index 0) is removed from the list.
    # This ensures that the memory never grows beyond its capacity, maintaining a fixed size and allowing newer experiences to replace older ones.


  def sample(self, batch_size): # This method is used to randomly sample a batch of experiences from the replay memory for training.
    experiences = random.sample(self.memory, k = batch_size)
    # random.sample is a Python standard library function that randomly selects batch_size elements from self.memory.
    # k=batch_size indicates the number of experiences to sample.
    # The sampled experiences are stored in the experiences list.

# extract individual components (states, actions, rewards, next states, and done flags) from the sampled experiences and convert them into PyTorch
# tensors for further processing.
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

# This part creates a NumPy array by vertically stacking (vstack) the first element (e[0]) of each experience in experiences.
# e[0] corresponds to the state in each experience tuple.
# The same approach is used for actions (e[1]), rewards (e[2]), next states (e[3]), and done flags (e[4]).
# torch.from_numpy(...) converts the NumPy array into a PyTorch tensor.
# .float() casts the tensor to a float32 type (except for actions, which are converted to long type).
# .to(self.device) moves the tensor to the specified device (GPU or CPU), ensuring the computations are done on the right hardware.
# states: A tensor containing all the states from the sampled experiences.
# actions: A tensor containing all the actions from the sampled experiences, cast to long because actions are typically integers.
# rewards: A tensor containing all the rewards from the sampled experiences.
# next_states: A tensor containing all the next states from the sampled experiences.
# dones: A tensor containing all the done flags from the sampled experiences, cast to float after being converted to uint8 (0 or 1).

### Implementing the DQN class

In [28]:
class Agent():
# This class is used in a reinforcement learning setup, most likely to train a deep Q-network (DQN) to play a game or solve a task.
# The agent interacts with the environment, learns from it, and improves its decision-making over time.
  def __init__(self, state_size, action_size): # This is the setup function for the Agent. It runs as soon as you create an Agent object.
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #self.device: This sets up where the agent will do its calculations—either on the GPU if available (faster) or the CPU (slower).
    self.state_size = state_size
    self.action_size = action_size
    # These store the size of the input (state) and the number of possible actions the agent can take.
    self.local_qnetwork = Network(state_size, action_size).to(self.device) #This is the main neural network the agent uses to predict the best actions. It takes the state as input and outputs action values.
    self.target_qnetwork = Network(state_size, action_size).to(self.device) #This is a copy of the local_qnetwork. It helps stabilize training by providing a stable set of action values to compare against. It's updated slowly to match the local network.
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate) # This is the algorithm that updates the weights of the local_qnetwork during learning. It uses the Adam optimizer, which is a popular method for training deep learning models.
    self.memory = ReplayMemory(replay_buffer_size) #This is the replay memory where the agent stores its experiences (state, action, reward, next state, done). The agent will sample from this memory to learn.
    self.t_step = 0 #This keeps track of the number of steps the agent has taken. It’s used to control when the agent updates its knowledge.

  def step(self, state, action, reward, next_state, done): #This function handles what happens each time the agent takes a step in the environment.
    self.memory.push((state, action, reward, next_state, done))#The agent saves its experience (what it did and what happened) in its memory.
    self.t_step = (self.t_step + 1) % 4 # This increments the step counter by 1 and checks if it’s divisible by 4. This means the agent only learns from its experiences every 4 steps.
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)
# Check Memory Size: The agent only learns if it has enough experiences in memory (minibatch_size).
# Sample Experiences: It randomly picks a batch of 100 experiences from memory.
# Call learn: The agent uses these experiences to update its knowledge.

  def act(self, state, epsilon = 0.): # This function decides which action the agent should take given the current state.
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) #Convert State to Tensor: The state is turned into a PyTorch tensor and sent to the appropriate device (CPU/GPU).
    self.local_qnetwork.eval() #Puts the network in evaluation mode (no training, just inference).
    with torch.no_grad(): #Temporarily stops tracking operations to save memory, since we don’t need gradients when just picking an action.
      action_values = self.local_qnetwork(state) # The local_qnetwork predicts the values of each possible action for the given state.
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))
# Epsilon-Greedy Strategy: This strategy balances exploration and exploitation.
# With probability 1 - epsilon, the agent picks the action with the highest predicted value (exploitation).
# With probability epsilon, the agent picks a random action (exploration).
# Return Chosen Action: The chosen action is returned for the agent to execute.

  def learn(self, experiences, discount_factor): # This is where the agent learns from its experiences to improve its decision-making.
    states, next_states, actions, rewards, dones = experiences # The experiences (states, actions, rewards, next states, and dones) are unpacked from the batch.
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1) #These are the best Q-values predicted by the target_qnetwork for the next states.
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones) #  These are the targets the local_qnetwork should aim for, calculated using the Bellman equation. The (1 - dones) part ensures that if the episode is done (the game is over), the future reward is not considered.
    q_expected = self.local_qnetwork(states).gather(1, actions) #These are the Q-values predicted by the local_qnetwork for the given states and actions. gather(1, actions) is used to pick the Q-values corresponding to the actions the agent actually took.
    loss = F.mse_loss(q_expected, q_targets) # The difference between the expected Q-values and the target Q-values is calculated using Mean Squared Error (MSE) loss.
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

# Zero Gradients: The gradients are reset before computing new ones.
# Compute Gradients: The loss is backpropagated through the network to compute the gradients.
# Update Weights: The optimizer updates the network’s weights to reduce the loss.
# Update Target Network: The target_qnetwork is slowly updated to match the local_qnetwork using soft updates.

  def soft_update(self, local_model, target_model, interpolation_parameter): #This method slowly updates the target_qnetwork to keep it close to the local_qnetwork without being exactly the same.
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

# Update Parameter: The target parameter is updated to be a mix of the local parameter and the existing target parameter.
# The interpolation_parameter controls how much of the local network is mixed into the target network.
# This slow update process helps stabilize learning by ensuring the target network doesn't change too rapidly, reducing the chance of oscillations or divergence in training.

### Initializing the DQN agent

In [29]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [30]:
number_episodes = 2000 # The total number of episodes (or rounds) the agent will play during training.
maximum_number_timesteps_per_episode = 1000 # The maximum number of actions the agent can take in a single episode before the episode ends automatically.
epsilon_starting_value  = 1.0 #The initial value of epsilon, controlling how often the agent will explore vs. exploit (starting with full exploration).
epsilon_ending_value  = 0.01 #The minimum value of epsilon, preventing the agent from completely stopping exploration.
epsilon_decay_value  = 0.995 #The rate at which epsilon decreases after each episode, gradually shifting the agent from exploration to exploitation.
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100) #A deque (double-ended queue) to keep track of the scores from the last 100 episodes, used to calculate a running average score.

for episode in range(1, number_episodes + 1): # This loop runs over each episode from 1 to number_episodes
  state, _ = env.reset() #Resets the environment to a starting state at the beginning of each episode. The agent receives the initial state.
  score = 0 #Initializes the score for this episode to 0. The score will accumulate as the agent collects rewards.
  for t in range(maximum_number_timesteps_per_episode): #This loop allows the agent to take up to maximum_number_timesteps_per_episode actions in a single episode.
    action = agent.act(state, epsilon) #The agent selects an action based on the current state and the epsilon value. With probability epsilon, the agent explores by taking a random action; otherwise, it exploits by choosing the action with the highest predicted value.
    next_state, reward, done, _, _ = env.step(action) #The environment takes the chosen action and returns:
# next_state: The state after the action.
# reward: The reward received for taking that action.
# done: A boolean indicating whether the episode has ended (e.g., the agent wins or loses the game).
    agent.step(state, action, reward, next_state, done) #The agent stores the experience in memory and possibly learns from a random batch of experiences (depending on self.t_step). This step is where the agent updates its neural network.
    state = next_state #The agent moves to the next state.
    score += reward #The agent's score for the episode is updated by adding the received reward.
    if done:
      break # If the environment indicates that the episode is finished (done is True), the loop breaks, and the episode ends early.
  scores_on_100_episodes.append(score) #The score for this episode is added to scores_on_100_episodes, which keeps track of the last 100 scores.
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon) # Epsilon is decayed after each episode, making the agent less likely to explore and more likely to exploit as training progresses. It is ensured that epsilon doesn’t go below epsilon_ending_value.
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0: #Every 100 episodes, the code prints out the average score more prominently on a new line.
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

#     if np.mean(scores_on_100_episodes) >= 200.0:: If the average score over the last 100 episodes reaches or exceeds 200, the environment is considered "solved."
# Save Model:
# torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth'): The agent's model is saved to a file called checkpoint.pth. This file can be used to reload the trained model later.
# break: The training loop is terminated early if the environment is solved.

Episode 100	Average Score: -159.55
Episode 200	Average Score: -98.32
Episode 300	Average Score: -54.03
Episode 400	Average Score: -16.04
Episode 500	Average Score: 82.26
Episode 600	Average Score: 138.41
Episode 700	Average Score: 113.46
Episode 773	Average Score: 202.22
Environment solved in 673 episodes!	Average Score: 202.22


## Part 3 - Visualizing the results

In [31]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()